# Annotation Bounding Boxes
*BeamNG.tech only*

This example shows how to use the camera sensor to get both semantic and instance annotations from the simulation and extract bounding box information from those two channels. The identified bounding boxes will then be drawn onto the regular color image of the camera and encoded as XML.

## Scenario

For our demonstration, we'll set up an intersection with a total of five cars on it, two facing the ego vehicle, and two going in the same direction as the ego vehicle. The cars will be placed such that they partially overlap each other, meaning they become inseparable in the semantic annotation as their class colors are the same, illustrating the use of instance annotations.

## Setup

The classes required in this example boil down to the following:

* `BeamNGpy`: The main class representing the simulator, used to control the simulation process from Python.
* `Scenario` : Used to specify the desired scenario, i.e. the level we want to use and which vehicles are participating.
* `Vehicle`: Class representing a vehicle. Our scenario will contain five `Vehicle` instances with different models, one being the ego vehicle we will get the perspective of and four being other cars crossing the intersection.
* `Camera`: The camera sensor used to get image data and annotation information from the simulator. In addition to the sensor simulation it also contains utility functions to process that data further, such as bounding box analysis and encoding.

In [1]:
%matplotlib inline

import time

import matplotlib.pyplot as plt
import numpy as np
from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Camera

For scenario creation we will instantiate the `Scenario` class with the desired map and scenario name as the respective parameters. Five instances of the `Vehicle` class are then created and added to the scenario at their respective positions. These positions were determined by manually loading the map and finding desired locations using the World Editor.

In [2]:
beamng = BeamNGpy('localhost', 64256)
beamng.open()

scenario = Scenario('italy', 'annotation_bounding_boxes')

ego = Vehicle('ego', model='etk800', color='White')
scenario.add_vehicle(ego, pos=(237.90, -894.42, 246.10),
                     rot_quat=(0.0173, -0.0019, -0.6354, 0.7720))

car1 = Vehicle('car1', model='etk800', color='Green')
scenario.add_vehicle(car1, pos=(246.94, -901.64, 247.58),
                     rot_quat=(-0.0099, 0.0206, 0.9348, -0.3543))

car2 = Vehicle('car2', model='etk800', color='Red')
scenario.add_vehicle(car2, pos=(276.27, -881.42, 247.84),
                     rot_quat=(-0.0106, 0.0405, 0.4845, 0.8738))

car3 = Vehicle('car3', model='etki', color='Blue')
scenario.add_vehicle(car3, pos=(261.52, -894.68, 248.04),
                     rot_quat=(0.0026, 0.01758, -0.8344, 0.5508))

car4 = Vehicle('car4', model='miramar', color='Black')
scenario.add_vehicle(car4, pos=(267.06, -892.03, 248.32),
                     rot_quat=(0.0065, 0.0194, -0.8501, 0.5262))

scenario.make(beamng)

The first vehicle added, as the name implies, is our ego-vehicle. It is also equipped with a camera sensor that is configured to return semantic and instance annotation, indicated by the `annotation` and `instance` flags. This means the captured data will contain at least three images: the regular color image rendered, an additional image where each pixel is colored according to the class of the object the pixel belongs to, and another image where each pixel's color is a unique color of the object it belongs to, i.e. even though multiple cars have the same class and thus the same color in the semantic annotations, they have a different color in the instance annotations.

Finally, the scenario is compiled and placed in a location where the simulator can find it by calling `scenario.make(beamng)` at the end. This means we can start the simulation and load our scenario.

## Running

We first start the simulator and then immediately tell it to load our scenario. Once it is loaded, we move on to capturing data from the camera sensor. This is used by calling `poll_sensors` on our ego-vehicle, which has a camera attached according to our scenario definition. For later processing we also retrieve the annotation color configuration of the simulator using the `bng.get_annotations()` function. This will be used later to identify what class of object a color refers to. After that, we can close the simulator as all further processing is done offline.

In [3]:
beamng.load_scenario(scenario)
beamng.start_scenario()
beamng.switch_vehicle(ego)
time.sleep(10)  # Some sleeping time to make sure the level fully loaded.

cam1 = Camera('camera1', beamng, ego, requested_update_time=-1.0, is_using_shared_memory=True, pos=(-0.3, 1, 2), dir=(0, -1, 0),
    field_of_view_y=70, near_far_planes=(0.1, 100), resolution=(1024, 1024), is_render_instance=True)

time.sleep(2)

images = cam1.get_full_poll_request()

ego.poll_sensors()
data = ego.sensors['camera'].data
annotations = beamng.get_annotations()

## Analysis

The camera sensor provided us with three captured images: regular color information, semantic annotation, and instance annotation. Each of these images can be seen below:

In [4]:
fig, ax = plt.subplots(1, 4, figsize=(30, 30))
ax[0].imshow(np.asarray(images['colour'].convert('RGB')))
ax[1].imshow(np.asarray(images['annotation'].convert('RGB')))
ax[2].imshow(np.asarray(images['instance'].convert('RGB')))
ax[3].imshow(np.asarray(images['depth'].convert('RGB')))

<Figure size 2160x2160 with 3 Axes>

With both semantic and instance annoations, we can use a utility function of the `Camera` sensor to extract bounding boxes of objects in the image. This is done with the `Camera.extract_bboxes` function. To map colors to object classes this function requires a respective dictionary. Since we obtained the annotation configuration earlier, we can generate such a dictionary using the `beamng.get_annotation_classes()` function. We then use that and the previously obtained images to extract bounding boxes of the image. This example will focus on vehicles seen by the camera, so we will narrow down the list to only bounding boxes of vehicles.

In [5]:
class_data = beamng.get_annotation_classes(annotations)
bounding_boxes = Camera.extract_bounding_boxes(images['annotation'], images['instance'], class_data)
bounding_boxes = [b for b in bounding_boxes if b['class'] == 'CAR']
print(bounding_boxes)

[{'bbox': [105, 475, 159, 502], 'class': 'CAR', 'color': [215, 61, 123, 255]},
 {'bbox': [757, 478, 1023, 590],
  'class': 'CAR',
  'color': [219, 186, 113, 255]},
 {'bbox': [285, 481, 355, 518], 'class': 'CAR', 'color': [222, 43, 149, 255]},
 {'bbox': [342, 486, 439, 529], 'class': 'CAR', 'color': [151, 58, 206, 255]},
 {'bbox': [0, 828, 1023, 1023], 'class': 'CAR', 'color': [45, 10, 210, 255]}]

The resulting list contains exactly five vehicles. Each entry in this list has the bounding box corners in pixel coordinates, the class of the object, and the color of the object in the instance annotations. This can now be used to draw bounding boxes onto our regular color image with the `Camera.draw_bboxes()` function.

In [6]:
image_with_boxes = Camera.draw_bounding_boxes(bounding_boxes, images['colour'], width=3)
plt.clf()
plt.figure(figsize=(15, 15))
plt.imshow(np.asarray(image_with_boxes.convert('RGB')))
plt.show()

<Figure size 432x288 with 0 Axes>

<Figure size 1080x1080 with 1 Axes>

The same information can also be encoded as Pascal-VOC XML with the `Camera.export_bbox_xml()` function:

In [7]:
xml = Camera.export_bounding_boxes_xml(bounding_boxes, filename='example.png', size=(1024, 1024, 3))
print(xml)

<?xml version="1.0" ?>
<annotation>
  <filename>example.png</filename>
  <size>
    <width>1024</width>
    <height>1024</height>
    <depth>3</depth>
  </size>
  <segmented>0</segmented>
  <object>
    <name>CAR_0</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
      <xmin>105</xmin>
      <ymin>475</ymin>
      <xmax>159</xmax>
      <ymax>502</ymax>
    </bndbox>
  </object>
  <object>
    <name>CAR_1</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
      <xmin>757</xmin>
      <ymin>478</ymin>
      <xmax>1023</xmax>
      <ymax>590</ymax>
    </bndbox>
  </object>
  <object>
    <name>CAR_2</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <bndbox>
      <xmin>285</xmin>
      <ymin>481</ymin>
      <xmax>355</xmax>
      <ymax>518</ymax>
    </bndbox>
  </object>
  <object>
    <name>CAR_3</name>
    <pose>Unspecified

In [ ]:
beamng.close()